In [1]:
from src.evaluate import Evaluate
from src.supply_chain_config import SupplyChainConfig
from src.policy import MeanDemandPolicy, StaticBaseStockPolicyRandom, StaticBaseStockPolicyShortfall, EchelonBaseStockPolicy , NonstationaryBaseStockPolicy, sSPolicy, DualIndexPolicy 
from src.dynamics import Dynamics

import numpy as np
import pandas as pd

In [2]:
def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Initialize network & a base strategy
</div>

In [3]:
# Create an instance of supply chain configuration
supply_chain_config = SupplyChainConfig()
dynamics = Dynamics(supply_chain_config)

# Policy parameters
hedging_base_stock_shortfall = 0.25
hedging_base_stock_random = 0.25
hedging_mean_demand_policy = 0.1
hedging_echelon = 0.25

# Creat a base policy
base_stock_policy_random = StaticBaseStockPolicyRandom(supply_chain_config, hedging_base_stock_random)
base_stock_levels = np.array(base_stock_policy_random.base_stock_levels)
echelon_policy = EchelonBaseStockPolicy(supply_chain_config, hedging_echelon)



<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Generate the data
</div>

In [5]:
from src.supply_chain_config import SupplyChainConfig
from src.dynamics import Dynamics
from src.evaluate import Evaluate
from src.policy import StaticBaseStockPolicyRandom, EchelonBaseStockPolicy
import numpy as np
import pandas as pd

def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

# 1. Initialize environment
sc = SupplyChainConfig()
dyn = Dynamics(sc)

# 2. Policy parameters
hedge_random  = 0.25
hedge_echelon = 0.25

# 3. Instantiate the two policies
policy_random  = StaticBaseStockPolicyRandom(sc, hedge_random)
policy_echelon = EchelonBaseStockPolicy( sc, hedge_echelon)

policies = [policy_random, policy_echelon]
num_policies = len(policies)

# 4. Simulation params
num_demand_patterns    = 10
num_trajectories       = 1
periods_per_trajectory = 200

evalr = Evaluate(sc, dyn, num_trajectories, periods_per_trajectory)

# 5. Run simulations and collect
data_parts = []
for dp in range(num_demand_patterns):
    costs = evalr.compare_policies(policies)
    # assemble base‑stock targets
    bs_arr = []
    for p in policies:
        if hasattr(p, "base_stock_levels"):
            bs_arr.append(p.base_stock_levels)
        else:
            bs_arr.append(p.S)
    bs_arr = np.array(bs_arr)
    block = np.concatenate([
        np.arange(num_policies).reshape(-1,1),
        np.ones((num_policies,1))*dp,
        bs_arr,
        np.array(costs).reshape(-1,1)
    ], axis=1)
    data_parts.append(block)

# 6. Build DataFrame & save
cols = ["policy_id", "demand_pattern"] + [f"C{i+1}" for i in range(sc.num_components)] + ["cost"]
df = pd.DataFrame(np.vstack(data_parts), columns=cols)
df.to_csv("data/generated_data.csv", index=False)

print(f"Results for {num_policies} policies across {num_demand_patterns} patterns → saved to data/generated_data.csv")



Results for 2 policies across 10 patterns → saved to data/generated_data.csv


In [ ]:

#CODE TIM



import numpy as np
import pandas as pd

from src.supply_chain_config import SupplyChainConfig
from src.dynamics import Dynamics
from src.evaluate import Evaluate
from src.policy import (
    MeanDemandPolicy,
    StaticBaseStockPolicyRandom,
    StaticBaseStockPolicyShortfall,
    EchelonBaseStockPolicy,
    NonstationaryBaseStockPolicy,
    sSPolicy,
    DualIndexPolicy,
)

# 1) Initialize environment
sc  = SupplyChainConfig()
dyn = Dynamics(sc)

# 2) Simulation parameters
num_trajectories       = 1000
periods_per_trajectory = 200

# 3) Precompute canonical S for sSPolicy and DualIndexPolicy
base_zero    = StaticBaseStockPolicyRandom(sc, hedging_parameter=0.0)
canonical_S  = np.array(base_zero.base_stock_levels, dtype=int)

# 4) Hedging / service‐level grid
hedges = np.arange(0.0, 1.01, 0.1)

# 5) Run experiments
results = []
for h in hedges:
    # clamp service level for Nonstationary in (ε, 1–ε)
    sl = min(max(h, 1e-3), 1 - 1e-3)
    
    policies = [
        ("MeanDemandPolicy",             MeanDemandPolicy(sc, h)),
        ("StaticBaseStockPolicyRandom",  StaticBaseStockPolicyRandom(sc, h)),
        ("StaticBaseStockPolicyShortfall", StaticBaseStockPolicyShortfall(sc, h)),
        ("EchelonBaseStockPolicy",       EchelonBaseStockPolicy(sc, h)),
        ("NonstationaryBaseStockPolicy", NonstationaryBaseStockPolicy(sc, sl)),
        ("sSPolicy",                     sSPolicy(
                                             sc,
                                             s_levels=(h * canonical_S).tolist(),
                                             S_levels=(np.ceil((1 + h) * canonical_S)).astype(int).tolist()
                                         )),
        ("DualIndexPolicy",              DualIndexPolicy(
                                             sc,
                                             s_lower=(h * canonical_S).tolist(),
                                             S_upper=(np.ceil((1 + h) * canonical_S)).astype(int).tolist()
                                         )),
    ]
    
    for name, policy in policies:
        traj_costs = []
        for _ in range(num_trajectories):
            state = dyn.get_initial_state()
            total = 0
            for _ in range(periods_per_trajectory):
                action = policy.set_action(state)
                state, cost = dyn.transition_dynamics(state, action)
                total += cost
            traj_costs.append(total / periods_per_trajectory)
        
        results.append({
            "policy":    name,
            "hedge":     round(h, 2),
            "mean_cost": np.mean(traj_costs),
            "std_cost":  np.std(traj_costs),
        })

# 6) Assemble into DataFrame
df = pd.DataFrame(results)
print(df)

# 7) Pivot for comparison
pivot = df.pivot(index="hedge", columns="policy", values="mean_cost")
print(pivot)





                            policy  hedge    mean_cost     std_cost
0                 MeanDemandPolicy    0.0  2253.213005  1793.925671
1      StaticBaseStockPolicyRandom    0.0  1259.571322   329.898239
2   StaticBaseStockPolicyShortfall    0.0  1133.540624   269.351124
3           EchelonBaseStockPolicy    0.0  3561.249521   439.588340
4     NonstationaryBaseStockPolicy    0.0  3999.836903   472.211453
..                             ...    ...          ...          ...
72  StaticBaseStockPolicyShortfall    1.0   606.963561    65.627182
73          EchelonBaseStockPolicy    1.0  2269.002354   821.973764
74    NonstationaryBaseStockPolicy    1.0  3173.039603   444.780192
75                        sSPolicy    1.0   686.824495   241.136431
76                 DualIndexPolicy    1.0   744.514753   185.480055

[77 rows x 4 columns]
policy  DualIndexPolicy  EchelonBaseStockPolicy  MeanDemandPolicy  \
hedge                                                               
0.0        86893.356823